In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, Bidirectional, LSTM
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


In [49]:
def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    print (int_to_note)

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(LSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        #print (notes)
        self.generate(notes)
        self.plot_loss()
        
    def generate(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        #print (int_to_note)
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        #print (predictions)
        pred_notes = [x*242+242 for x in predictions[0]]
        #print (pred_notes)
        notess = []
        for x in pred_notes:
            if int(x)>=len(int_to_note):
                notess.append(len(int_to_note)-1)
            else:
                notess.append(x)
        pred_notes = [int_to_note[int(x)] for x in notess]
        
        create_midi(pred_notes, 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [51]:
gan = GAN(rows=100)    
gan.train(epochs=5, batch_size=20, sample_interval=1)

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 100, 512)          1052672   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 1024)              4198400   
_________________________________________________________________
dense_92 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_66 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_67 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_94 (Dense)             (None, 1)               

/home/bikash/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.693387, acc.: 50.00%] [G loss: 0.688721]


/home/bikash/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.677536, acc.: 77.50%] [G loss: 0.683964]
2 [D loss: 0.640956, acc.: 82.50%] [G loss: 0.672194]
3 [D loss: 0.553705, acc.: 82.50%] [G loss: 0.738159]
4 [D loss: 0.485589, acc.: 87.50%] [G loss: 0.856678]


In [25]:
def generate( input_notes):
    # Get pitch names and store in a dictionary
    notes = input_notes
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
    # Use random noise to generate sequences
    #noise = np.random.normal(0, 1, (1, self.latent_dim))
    predictions = [[0.2424,0.224]]
        
    pred_notes = [x*242+242 for x in predictions[0]]
    print (int(x))
    pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
    #create_midi(pred_notes, 'gan_final')

In [26]:
input_notes = ['B2', 'D2', 'D2', 'E2', 'D2', 'D2', 'E2', 'D2', 'D3', 'C3', 'E2', 'D2', 'B2', 'D2', '11.0', 'D2', 'D2', '7.0', 'D2', '7.0', '7.0', '10.0.2', 'D2', '10.0', 'D2', 'D2', '0.4', 'D2', '0.4', 'C3', '11.0', 'D2', '0.1', 'C3', 'E2', 'C2', 'C3', 'C2', 'C2', 'E2', '2.8', 'D2', 'E2', 'B2', 'D2', 'B-3', 'C2', '2.7', '2.7', '2.7', '11.4', '2.8', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'C#2', 'E2', 'C3', 'D2', 'C#2', 'C3', 'B2', 'D2', 'E2', '8.0', 'B2', 'D2', '4.8', 'C3', 'G2', 'B-3', 'D3', 'D2', 'D2', 'E2', 'D2', 'D2', 'E2', 'D3', 'C3', 'C#2', 'A2', 'D2', 'E2', 'C3', 'E2', 'C3', 'G2', 'G3', 'G#2', '11.0.4', '11.2', '0.2', 'C#2', 'C3', 'B2', '0.4', 'D2', 'B2', 'D2', '0.2', 'D2', '0.4', 'D3', '11.0', 'D2', '0.2', 'B2', 'C2', 'C3', 'C3', '11.0', 'D2', '0.2', '1.2', '7.0', 'G2', '7.0', 'F#2', '0.2', 'B-0', 'D2', '10.0', 'D2', 'B-0', '0.1', 'D2', 'B-0', '0.2', '0.6', 'F#2', 'C2', '10.0', '10.0', 'D2', 'B-0', 'C2', '10.0', 'D2', 'B-0', '0.2', '10.0', 'C3', 'D3', 'C3', 'C#2', 'E2', 'E2', 'G#2', 'C3', 'D3', 'D3', 'D2', 'E2', 'D2', 'A2', 'G2', '10.11', '11.2', '11.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', 'G#2', 'D2', 'E2', 'D2', 'E2', 'D2', '0.6', 'B-0', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '0.6', 'D2', 'B-0', '0.4.8', '2.6', 'F#2', '10.0', 'B-0', '8.0.2', 'B-0', 'D2', '0.6', 'B-0', '0.1', 'B-0', 'F#2', '0.6', 'F#2', 'B-0', '8.0', 'D2', 'B-0', 'F#2', 'C2', 'F#2', 'F#2', 'B-0', '10.0', '10.0', 'B-2', 'C2', 'D2', 'E2', 'C3', '8.11', 'D2', 'D2', 'C2', 'F#2', '7.0', 'D1', '2.8', 'C2', 'B-0', '0.6', 'F#2', 'B-0', '0.2', 'F#2', 'B-0', 'B-0', 'C2', 'B-0', '0.4', 'B-0', 'B-0', '10.0', 'E2', 'D2', '8.0.2', 'D2', 'D2', 'C2', '7.0', 'B-0', 'B-0', '0.4', 'D2', 'B-0', '10.0', '0.6', 'F#2', 'B-0', '0.2', 'B-0', 'F#2', '0.6', 'F#2', 'B-0', '0.4.8', 'D2', 'D1', '8.0', 'D2', 'C#2', 'E2', 'E2', '0.4.8', 'D2', 'D2', 'C2', '7.0', 'E-3', 'G3', 'G#2', 'F3', 'F3', 'G3', 'C#3', '5.8', 'C#3', 'E-3', 'G3', 'G#2', 'E-3', 'F3', 'G3', 'C#3', '3.8', 'C#3', 'E-3', 'C#3', '3.8', 'F3', 'E-3', 'E-3', '5.8', 'E-3', 'E-3', 'E-3', 'G#2', 'E-3', 'F3', 'E-3', 'E-3', '7.0', '0.3', '7.0', 'D2', '0.2.7', '0.2.7', '0.2.7', '10.0.2', '10.0.1', 'D2', 'D2', 'D2', 'D2', 'E2', 'G#2', 'D2', 'E2', 'E2', 'G#2', 'D3', 'D2', '7.0', 'B-0', '0.4', 'B-0', '0.6', 'F#2', 'B-0', '0.4', 'F#2', 'B-0', 'B-0', '0.6', 'B-0', 'B-0', '0.4', 'B-0', '10.0', '10.0', 'B-0', 'F#2', 'B-0', '0.4', 'B-0', 'F#2', '0.6', 'B-0', 'B-0', 'B-0', '0.4', 'D2', 'B-0', 'B-0', '0.6', '10.0', 'F#2', 'F#2', 'F#2', '0.4', 'F#2', 'B-0', '0.6', '10.0', 'B-0', 'B-0', 'F#2', '0.4', 'D1', 'C2', 'D1', 'D2', '0.4.8', 'D2', '0.4', 'D2', '0.4.8', 'D3', 'B2', 'C2', 'G3', '7.0', 'B-0', '0.4', 'B-0', 'B-0', '0.2', '10.0', 'B-0', 'F#2', 'B-0', '0.4', 'F#2', 'B-0', 'B-0', '10.0', 'B-0', 'B-0', 'B-0', '0.4', 'B-0', 'B-0', '10.0', '10.0', 'D1', '0.2', 'D2', '8.0', 'D2', 'C#2', '4.8', 'B-2', '7.8.0', 'B-0', 'B-0', '0.4', 'B-0', 'B-0', 'B-0', '10.0', 'B-0', 'B-0', 'F#2', '0.4', 'B-0', 'B-0', 'B-0', '10.0', 'B-0', 'B-0', 'B-0', '0.4', 'B-0', 'D1', 'D2', '4.8', 'D2', 'D2', '4.8', 'D2', '4.8', 'D3', 'B-3', 'C2', 'G3', '11.0', '2.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', 'E-3', '0.4.8', 'D2', 'E-3', 'F#2', '7.0', 'E-3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'B-0', 'E-3', '1.3.7', '0.4.8', 'D2', 'F3', '0.6', '0.3', 'F#2', 'E-3', '0.4.8', 'D2', 'F3', '10.0', '7.0', 'E-3', '7.0', '11.0', 'E-3', '10.0', '8.10.0.1.2.3', 'E-3', '4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', '8.0.2', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'E-3', '0.4.8', 'D2', 'F3', 'F#2', 'C2', '3.8', 'B-2', '0.5', 'E-3', '4.8', 'D2', 'B2', 'F#2', '7.0', 'F#2', 'F3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'E-3', '4.8', 'A2', 'A2', 'A2', '11.0', '7.0', '4.5.8.0', '7.0', 'D2', '0.2.3', '11.0', 'D2', '0.3', '0.3', 'B3', 'D2', '0.2.7', 'D1', 'C2', 'B-2', '7.0', 'C2', '7.0', '0.2', '11.0', '11.0', '11.0', '7.0', '0.2', '11.0', '7.0', '0.2', '11.0', '7.10.0', '10.0.3', '11.0.2', '11.0.2', '11.0.2', '10.11.0', '11.0.2', '8.11.0.2', 'D2', 'D2', 'D2', 'D2', 'E2', 'E2', '2.8', 'D2', 'E2', 'C3', 'B2', 'D2', 'E2', 'C3', 'G2', 'G2', '8.11.0', 'E-3', '0.4.8', 'F3', 'F#2', '0.3', 'B-0', 'F3', '4.8', 'C2', 'D2', 'F3', 'F#2', '0.3', 'B-0', 'F3', 'C2', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'B-0', 'E-3', '0.3', '0.4.8', 'D2', 'C2', 'E-3', 'F#2', '7.0', 'B-0', 'B-0', '0.2', 'D2', 'D1', 'B-0', 'C2', 'F#2', 'B-0', 'B-0', '0.4', 'D2', 'B-0', 'B-0', 'G#2', 'E2', 'D2', 'E2', 'D2', '7.0', '2.3.7', '4.8', 'B-0', 'D2', '10.0', 'D2', 'B-0', '0.4', 'B-0', 'C2', 'B-0', '0.4', 'D2', 'B-0', 'D2', '0.6', 'B-0', '0.4', '10.2', '8.0', 'D2', 'B-0', '0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', 'C2', '0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', '0.4', 'D2', 'B-0', 'C2', 'D2', '0.6', '0.2', 'D2', 'C2', 'D2', 'E2', 'E2', 'D2', '7.0', 'D2', 'D1', '0.4.8', 'D2', 'B-0', '0.2', '0.6', 'D2', 'B-0', '0.4.8', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', '0.4.8', 'B-0', '0.2', '0.6', 'D2', '0.2', '8.0', 'C#2', 'E2', '7.0', 'B-0', '0.4.8', 'B-0', '0.6', 'F#2', 'D1', '0.4.8', 'B-0', 'B-0', '0.6', '0.2', 'B-2', '0.4.8', '0.4', 'D1', 'G#2', '0.2', 'D2', 'E2', 'C2', 'B2', '0.2', 'E2', 'D2', '7.0', 'B-0', '0.4.8', 'B-0', 'B-0', '0.6', 'F#2', 'B-0', '0.4', 'B-0', 'F#2', '0.6', 'B-0', '0.4', 'B-0', 'B-0', 'B-0', '0.6', 'F#2', 'D1', 'B-2', '8.0', 'D2', 'E2', 'E2', 'D2', '8.11.0', 'F#2', 'G3', '0.4', 'F#2', 'B-0', 'B-0', '0.6', 'B-0', 'B-0', 'F#2', '0.4', 'B-0', 'B-0', '10.0', 'G2', '2.7', '2.7', '2.4', 'D2', 'C3', 'C3', 'C3', 'D3', 'E2', 'D2', 'C2', 'D3', 'C3', 'C2', 'C#2', 'D2', 'C2', 'B2', 'D2', 'D2', 'E2', 'D2', 'C2', 'C2', 'B2', 'E2', 'D2', 'D2', 'E2', 'B-0', 'B2', '2.3', '11.2', '11.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', 'D2', 'D2', 'D2', 'D2', 'D3', 'E2', 'D2', 'E2', 'D3', 'B2', 'B2', 'G2', 'G2', 'B-3', 'G2', 'G#2', 'G#2', 'G#2', 'D2', 'G#2', 'E2', 'D2', 'C#2', 'E2', 'G#2', 'E2', 'D2', '7.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'B-0', '0.4', '10.2', 'F#2', 'C2', 'D2', 'D1', '4.8', 'C2', '10.2', 'B-0', '10.0', 'D2', 'B-0', '0.4', '10.2', 'F#2', 'C2', 'D2', 'B-0', '10.0.4', 'D2', 'B-0', 'D2', '10.0', 'D2', '0.2', 'D2', '8.0.2', 'E2', '0.2', 'C3', '0.4', 'D2', '11.0', 'C3', '0.4', 'D2', '7.0', 'G2', '11.0', 'B-2', '0.4.8', '10.2', 'F#2', '0.6', 'D2', 'D1', '0.4.8', '10.2', 'F#2', 'C2', 'D2', 'D1', '4.8', 'C2', '10.2', 'F#2', '0.6', 'D2', 'D1', '8.0', 'E2', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', '10.1', 'G#2', '0.6', 'D2', 'D1', '0.4.8', '0.2', '10.0', 'E2', '0.4', 'C2', 'D2', '0.4', 'D2', '8.0.2', 'D2', '0.4', 'C3', '11.0', 'D2', 'C#2', '7.0', '11.0', 'D2', 'B-0', '0.4.8', '2', 'G#2', '0.6', 'D2', 'D1', '0.4.8', '2', '0.6', 'D2', 'D1', '4.8', 'C2', '2', 'F#2', 'C2', 'F#2', 'D2', 'D1', '4.8', 'C2', '10.2', 'B-0', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'B-0', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.2.4', '0.2.4', 'D2', '8.0.2', 'D2', 'C2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', '7.0', 'B-2', '4.6.8.0', 'D2', 'B-0', 'D2', '0.6', 'D2', 'D1', '0.4.8', 'B-0', 'D2', '0.6', 'D2', 'D1', '0.4.8', 'F#2', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', 'F#2', 'C2', '10.2', 'F#2', '10.0', 'D2', 'D1', '8.10.0.4', 'C2', '10.2', 'D2', '10.0', 'D2', 'D2', '0.2', 'D2', '4.8.10', 'C2', '0.2', 'D1', 'D2', 'D2', '8.10.0', 'D2', '0.2', '0.2.4', '0.1.2', 'D2', '0.2', 'D2', '0.2', 'C3', '7.0', 'G2', '7.0', 'D2', 'D2', '8.11.0', 'D2', 'D1', '0.4.8', '2', 'B-0', '0.6', 'D2', '2', '0.4.8', 'F#2', 'D2', 'D1', 'D2', '10.0', 'D2', '0.2', 'D1', '4.6.8.0', 'D2', 'D1', 'D2', '6.8.0', 'D2', 'B-0', '8.10.0.4', 'D2', 'B-0', '10.0', 'D2', 'D1', '8.10.0.4', '0.2', 'B-0', '2.6', '0.6', 'D2', 'D1', '8.10.0.4', 'D2', 'D1', 'D2', '0.2', '0.2', 'D2', 'C3', 'D2', 'E2', 'D2', 'C3', 'D2', 'E2', 'E2', 'E2', 'D2', 'G2', 'G#2', 'C3', '11.0', 'B-2', '8.10.0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'D2', 'D1', '0.4.8', 'B-0', 'D2', 'B-0', 'D2', 'D2', '0.6', 'D2', 'D1', 'B-2', 'G#2', '10.0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', '10.0.4', 'D2', '0.2', 'D2', '8.0.2', 'D2', 'D2', '0.4', 'D2', 'D2', '0.4', 'E2', '0.2', 'D2', '0.4', 'C3', '11.0', 'D2', '0.4', 'G#2', 'C3', 'G2', 'C2', 'G3']

In [38]:
a = ['B2', 'D2', 'D2', 'E2', 'D2', 'D2', 'E2', 'D2', 'D3', 'C3', 'E2', 'D2', 'B2', 'D2', '11.0', 'D2', 'D2', '7.0', 'D2', '7.0', '7.0', '10.0.2', 'D2', '10.0', 'D2', 'D2', '0.4', 'D2', '0.4', 'C3', '11.0', 'D2', '0.1', 'C3', 'E2', 'C2', 'C3', 'C2', 'C2', 'E2', '2.8', 'D2', 'E2', 'B2', 'D2', 'B-3', 'C2', '2.7', '2.7', '2.7', '11.4', '2.8', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'C#2', 'E2', 'C3', 'D2', 'C#2', 'C3', 'B2', 'D2', 'E2', '8.0', 'B2', 'D2', '4.8', 'C3', 'G2', 'B-3', 'D3', 'D2', 'D2', 'E2', 'D2', 'D2', 'E2', 'D3', 'C3', 'C#2', 'A2', 'D2', 'E2', 'C3', 'E2', 'C3', 'G2', 'G3', 'G#2', '11.0.4', '11.2', '0.2', 'C#2', 'C3', 'B2', '0.4', 'D2', 'B2', 'D2', '0.2', 'D2', '0.4', 'D3', '11.0', 'D2', '0.2', 'B2', 'C2', 'C3', 'C3', '11.0', 'D2', '0.2', '1.2', '7.0', 'G2', '7.0', 'F#2', '0.2', 'B-0', 'D2', '10.0', 'D2', 'B-0', '0.1', 'D2', 'B-0', '0.2', '0.6', 'F#2', 'C2', '10.0', '10.0', 'D2', 'B-0', 'C2', '10.0', 'D2', 'B-0', '0.2', '10.0', 'C3', 'D3', 'C3', 'C#2', 'E2', 'E2', 'G#2', 'C3', 'D3', 'D3', 'D2', 'E2', 'D2', 'A2', 'G2', '10.11', '11.2', '11.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', 'G#2', 'D2', 'E2', 'D2', 'E2', 'D2', '0.6', 'B-0', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '0.6', 'D2', 'B-0', '0.4.8', '2.6', 'F#2', '10.0', 'B-0', '8.0.2', 'B-0', 'D2', '0.6', 'B-0', '0.1', 'B-0', 'F#2', '0.6', 'F#2', 'B-0', '8.0', 'D2', 'B-0', 'F#2', 'C2', 'F#2', 'F#2', 'B-0', '10.0', '10.0', 'B-2', 'C2', 'D2', 'E2', 'C3', '8.11', 'D2', 'D2', 'C2', 'F#2', '7.0', 'D1', '2.8', 'C2', 'B-0', '0.6', 'F#2', 'B-0', '0.2', 'F#2', 'B-0', 'B-0', 'C2', 'B-0', '0.4', 'B-0', 'B-0', '10.0', 'E2', 'D2', '8.0.2', 'D2', 'D2', 'C2', '7.0', 'B-0', 'B-0', '0.4', 'D2', 'B-0', '10.0', '0.6', 'F#2', 'B-0', '0.2', 'B-0', 'F#2', '0.6', 'F#2', 'B-0', '0.4.8', 'D2', 'D1', '8.0', 'D2', 'C#2', 'E2', 'E2', '0.4.8', 'D2', 'D2', 'C2', '7.0', 'E-3', 'G3', 'G#2', 'F3', 'F3', 'G3', 'C#3', '5.8', 'C#3', 'E-3', 'G3', 'G#2', 'E-3', 'F3', 'G3', 'C#3', '3.8', 'C#3', 'E-3', 'C#3', '3.8', 'F3', 'E-3', 'E-3', '5.8', 'E-3', 'E-3', 'E-3', 'G#2', 'E-3', 'F3', 'E-3', 'E-3', '7.0', '0.3', '7.0', 'D2', '0.2.7', '0.2.7', '0.2.7', '10.0.2', '10.0.1', 'D2', 'D2', 'D2', 'D2', 'E2', 'G#2', 'D2', 'E2', 'E2', 'G#2', 'D3', 'D2', '7.0', 'B-0', '0.4', 'B-0', '0.6', 'F#2', 'B-0', '0.4', 'F#2', 'B-0', 'B-0', '0.6', 'B-0', 'B-0', '0.4', 'B-0', '10.0', '10.0', 'B-0', 'F#2', 'B-0', '0.4', 'B-0', 'F#2', '0.6', 'B-0', 'B-0', 'B-0', '0.4', 'D2', 'B-0', 'B-0', '0.6', '10.0', 'F#2', 'F#2', 'F#2', '0.4', 'F#2', 'B-0', '0.6', '10.0', 'B-0', 'B-0', 'F#2', '0.4', 'D1', 'C2', 'D1', 'D2', '0.4.8', 'D2', '0.4', 'D2', '0.4.8', 'D3', 'B2', 'C2', 'G3', '7.0', 'B-0', '0.4', 'B-0', 'B-0', '0.2', '10.0', 'B-0', 'F#2', 'B-0', '0.4', 'F#2', 'B-0', 'B-0', '10.0', 'B-0', 'B-0', 'B-0', '0.4', 'B-0', 'B-0', '10.0', '10.0', 'D1', '0.2', 'D2', '8.0', 'D2', 'C#2', '4.8', 'B-2', '7.8.0', 'B-0', 'B-0', '0.4', 'B-0', 'B-0', 'B-0', '10.0', 'B-0', 'B-0', 'F#2', '0.4', 'B-0', 'B-0', 'B-0', '10.0', 'B-0', 'B-0', 'B-0', '0.4', 'B-0', 'D1', 'D2', '4.8', 'D2', 'D2', '4.8', 'D2', '4.8', 'D3', 'B-3', 'C2', 'G3', '11.0', '2.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', 'E-3', '0.4.8', 'D2', 'E-3', 'F#2', '7.0', 'E-3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'B-0', 'E-3', '1.3.7', '0.4.8', 'D2', 'F3', '0.6', '0.3', 'F#2', 'E-3', '0.4.8', 'D2', 'F3', '10.0', '7.0', 'E-3', '7.0', '11.0', 'E-3', '10.0', '8.10.0.1.2.3', 'E-3', '4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'F3', '8.0.2', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'E-3', '0.4.8', 'D2', 'F3', 'F#2', 'C2', '3.8', 'B-2', '0.5', 'E-3', '4.8', 'D2', 'B2', 'F#2', '7.0', 'F#2', 'F3', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'F#2', 'E-3', '4.8', 'A2', 'A2', 'A2', '11.0', '7.0', '4.5.8.0', '7.0', 'D2', '0.2.3', '11.0', 'D2', '0.3', '0.3', 'B3', 'D2', '0.2.7', 'D1', 'C2', 'B-2', '7.0', 'C2', '7.0', '0.2', '11.0', '11.0', '11.0', '7.0', '0.2', '11.0', '7.0', '0.2', '11.0', '7.10.0', '10.0.3', '11.0.2', '11.0.2', '11.0.2', '10.11.0', '11.0.2', '8.11.0.2', 'D2', 'D2', 'D2', 'D2', 'E2', 'E2', '2.8', 'D2', 'E2', 'C3', 'B2', 'D2', 'E2', 'C3', 'G2', 'G2', '8.11.0', 'E-3', '0.4.8', 'F3', 'F#2', '0.3', 'B-0', 'F3', '4.8', 'C2', 'D2', 'F3', 'F#2', '0.3', 'B-0', 'F3', 'C2', '0.4.8', 'D2', 'F3', 'F#2', '0.3', 'B-0', 'E-3', '0.3', '0.4.8', 'D2', 'C2', 'E-3', 'F#2', '7.0', 'B-0', 'B-0', '0.2', 'D2', 'D1', 'B-0', 'C2', 'F#2', 'B-0', 'B-0', '0.4', 'D2', 'B-0', 'B-0', 'G#2', 'E2', 'D2', 'E2', 'D2', '7.0', '2.3.7', '4.8', 'B-0', 'D2', '10.0', 'D2', 'B-0', '0.4', 'B-0', 'C2', 'B-0', '0.4', 'D2', 'B-0', 'D2', '0.6', 'B-0', '0.4', '10.2', '8.0', 'D2', 'B-0', '0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', 'C2', '0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', '0.4', 'D2', 'B-0', 'C2', 'D2', '0.6', '0.2', 'D2', 'C2', 'D2', 'E2', 'E2', 'D2', '7.0', 'D2', 'D1', '0.4.8', 'D2', 'B-0', '0.2', '0.6', 'D2', 'B-0', '0.4.8', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', '0.4.8', 'B-0', '0.2', '0.6', 'D2', '0.2', '8.0', 'C#2', 'E2', '7.0', 'B-0', '0.4.8', 'B-0', '0.6', 'F#2', 'D1', '0.4.8', 'B-0', 'B-0', '0.6', '0.2', 'B-2', '0.4.8', '0.4', 'D1', 'G#2', '0.2', 'D2', 'E2', 'C2', 'B2', '0.2', 'E2', 'D2', '7.0', 'B-0', '0.4.8', 'B-0', 'B-0', '0.6', 'F#2', 'B-0', '0.4', 'B-0', 'F#2', '0.6', 'B-0', '0.4', 'B-0', 'B-0', 'B-0', '0.6', 'F#2', 'D1', 'B-2', '8.0', 'D2', 'E2', 'E2', 'D2', '8.11.0', 'F#2', 'G3', '0.4', 'F#2', 'B-0', 'B-0', '0.6', 'B-0', 'B-0', 'F#2', '0.4', 'B-0', 'B-0', '10.0', 'G2', '2.7', '2.7', '2.4', 'D2', 'C3', 'C3', 'C3', 'D3', 'E2', 'D2', 'C2', 'D3', 'C3', 'C2', 'C#2', 'D2', 'C2', 'B2', 'D2', 'D2', 'E2', 'D2', 'C2', 'C2', 'B2', 'E2', 'D2', 'D2', 'E2', 'B-0', 'B2', '2.3', '11.2', '11.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', '11.0.2', 'D2', 'D2', 'D2', 'D2', 'D3', 'E2', 'D2', 'E2', 'D3', 'B2', 'B2', 'G2', 'G2', 'B-3', 'G2', 'G#2', 'G#2', 'G#2', 'D2', 'G#2', 'E2', 'D2', 'C#2', 'E2', 'G#2', 'E2', 'D2', '7.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'B-0', '0.4', '10.2', 'F#2', 'C2', 'D2', 'D1', '4.8', 'C2', '10.2', 'B-0', '10.0', 'D2', 'B-0', '0.4', '10.2', 'F#2', 'C2', 'D2', 'B-0', '10.0.4', 'D2', 'B-0', 'D2', '10.0', 'D2', '0.2', 'D2', '8.0.2', 'E2', '0.2', 'C3', '0.4', 'D2', '11.0', 'C3', '0.4', 'D2', '7.0', 'G2', '11.0', 'B-2', '0.4.8', '10.2', 'F#2', '0.6', 'D2', 'D1', '0.4.8', '10.2', 'F#2', 'C2', 'D2', 'D1', '4.8', 'C2', '10.2', 'F#2', '0.6', 'D2', 'D1', '8.0', 'E2', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', '10.1', 'G#2', '0.6', 'D2', 'D1', '0.4.8', '0.2', '10.0', 'E2', '0.4', 'C2', 'D2', '0.4', 'D2', '8.0.2', 'D2', '0.4', 'C3', '11.0', 'D2', 'C#2', '7.0', '11.0', 'D2', 'B-0', '0.4.8', '2', 'G#2', '0.6', 'D2', 'D1', '0.4.8', '2', '0.6', 'D2', 'D1', '4.8', 'C2', '2', 'F#2', 'C2', 'F#2', 'D2', 'D1', '4.8', 'C2', '10.2', 'B-0', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'B-0', '10.0', 'D2', 'D1', '0.4.8', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.2.4', '0.2.4', 'D2', '8.0.2', 'D2', 'C2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', '7.0', 'B-2', '4.6.8.0', 'D2', 'B-0', 'D2', '0.6', 'D2', 'D1', '0.4.8', 'B-0', 'D2', '0.6', 'D2', 'D1', '0.4.8', 'F#2', '10.2', 'F#2', '10.0', 'D2', 'D1', '0.4.8', 'F#2', 'C2', '10.2', 'F#2', '10.0', 'D2', 'D1', '8.10.0.4', 'C2', '10.2', 'D2', '10.0', 'D2', 'D2', '0.2', 'D2', '4.8.10', 'C2', '0.2', 'D1', 'D2', 'D2', '8.10.0', 'D2', '0.2', '0.2.4', '0.1.2', 'D2', '0.2', 'D2', '0.2', 'C3', '7.0', 'G2', '7.0', 'D2', 'D2', '8.11.0', 'D2', 'D1', '0.4.8', '2', 'B-0', '0.6', 'D2', '2', '0.4.8', 'F#2', 'D2', 'D1', 'D2', '10.0', 'D2', '0.2', 'D1', '4.6.8.0', 'D2', 'D1', 'D2', '6.8.0', 'D2', 'B-0', '8.10.0.4', 'D2', 'B-0', '10.0', 'D2', 'D1', '8.10.0.4', '0.2', 'B-0', '2.6', '0.6', 'D2', 'D1', '8.10.0.4', 'D2', 'D1', 'D2', '0.2', '0.2', 'D2', 'C3', 'D2', 'E2', 'D2', 'C3', 'D2', 'E2', 'E2', 'E2', 'D2', 'G2', 'G#2', 'C3', '11.0', 'B-2', '8.10.0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'D2', 'D1', '0.4.8', 'B-0', 'D2', 'B-0', 'D2', 'D2', '0.6', 'D2', 'D1', 'B-2', 'G#2', '10.0.4', 'D2', 'B-0', 'D2', '0.6', 'D2', 'B-0', '10.0.4', 'D2', '0.2', 'D2', '8.0.2', 'D2', 'D2', '0.4', 'D2', 'D2', '0.4', 'E2', '0.2', 'D2', '0.4', 'C3', '11.0', 'D2', '0.4', 'G#2', 'C3', 'G2', 'C2', 'G3']

In [39]:
a[300]

'F3'

In [27]:
generate(input_notes)


[300.6608, 296.20799999999997]


KeyError: 300